In [1]:
import sys
sys.path.append('../misc')
from helpers import *
import pandas as pd
df = get_full_table('../data/training_variants','../data/training_text')
df2 = get_full_table('../data/test_variants','../data/test_text')
df3 = get_full_table('../data/stage2_test_variants.csv', '../data/stage2_test_text.csv')
new_df1 = pd.concat([df, df2], axis=0)
new_df2 = pd.concat([df, df2, df3], axis=0)

gene1 = list(new_df1['Gene'].unique())
gene2 = list(new_df2['Gene'].unique())
gene = [i for i in gene2 if i not in gene1]
len(gene)

14

In [2]:
def get_from_path(path):
    """given a request path and return the 
    response and content"""
    import httplib2 as http
    import json
    try:
        from urlparse import urlparse
    except ImportError:
        from urllib.parse import urlparse

    headers = {
     'Accept': 'application/json',
    }

    uri = 'http://rest.genenames.org'

    target = urlparse(uri+path)
    method = 'GET'
    body = ''

    h = http.Http()

    response, content = h.request(
     target.geturl(),
     method,
     body,
     headers)
    return response, content
  
def get_gene_json(gene):
    """given gene, return a dict with info about it"""
    import time
    
    path = '/fetch/symbol/'+gene
    response, content = get_from_path(path)    
    if response['status'] == '200':
        # assume that content is a json reply
        # parse content with the json module 
        data = json.loads(content)
        #print(type(data))
        #print('Symbol:' + data['response']['docs'][0]['symbol'])
        #print('Symbol:' + data['response']['docs'][0]['name'])
        if data['response']['numFound'] != 0:
            print(data['response']['numFound'])
            return data
        else:
            path = '/fetch/prev_symbol/'+gene
            time.sleep(3)
            response, content = get_from_path(path)
            if response['status'] == '200':
                data = json.loads(content)
                if data['response']['numFound'] != 0:
                    return data
            else:
                print('Error detected: ' + response['status'])
                
    else:
        print('Error detected: ' + response['status'])
    

In [3]:
import time
import json
fail_list=[]
json_dict={}
for i, g in enumerate(gene):
    data = get_gene_json(g)
    if data == None:
        fail_list.append(g)
    else:
        json_dict[g] = data
    time.sleep(3)
    
#    if i%100 == 0:
#        with open('result'+str(i)+'.json', 'w') as f:
#            json.dump(json_dict, f)
            

1
1
1
1
1
1
1
1
1
1
1
1
1
1


FileNotFoundError: [Errno 2] No such file or directory: './input/result.json'

In [4]:
with open('../feature_engineer/result.json', 'w') as f:
            json.dump(json_dict, f)  

In [5]:
with open('../feature_engineer/result/gene_query_result_old.json','r') as f:
    d = json.load(f)
with open('../feature_engineer/result.json','r') as f:
    d2 = json.load(f)

In [6]:
new_d = dict(d)
new_d.update(d2)

In [7]:
with open('../feature_engineer/gene_query_result.json', 'w') as f:
    json.dump(new_d, f)